In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5150
False    4850
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [7]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [9]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
39,1,32,1,41.0
6132,0,28,1,30.0
5986,0,44,1,38.0
7682,0,34,1,38.0
4910,1,31,1,40.0


In [10]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [11]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [12]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [13]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6670 samples, validate on 742 samples
Epoch 1/10
6670/6670 [==============================] - 2s 338us/sample - loss: 1.9177 - mae: 1.0818 - mse: 1.9177 - val_loss: 1.4019 - val_mae: 0.9318 - val_mse: 1.4019
Epoch 2/10
6670/6670 [==============================] - 1s 158us/sample - loss: 1.6174 - mae: 0.9992 - mse: 1.6174 - val_loss: 2.0704 - val_mae: 1.1461 - val_mse: 2.0704
Epoch 3/10
6670/6670 [==============================] - 1s 149us/sample - loss: 1.5341 - mae: 0.9749 - mse: 1.5341 - val_loss: 1.3862 - val_mae: 0.9285 - val_mse: 1.3862
Epoch 4/10
6670/6670 [==============================] - 1s 156us/sample - loss: 1.4332 - mae: 0.9393 - mse: 1.4332 - val_loss: 2.7086 - val_mae: 1.3735 - val_mse: 2.7086
Epoch 5/10
6670/6670 [==============================] - 1s 174us/sample - loss: 1.3805 - mae: 0.9229 - mse: 1.3805 - val_loss: 2.1895 - val_mae: 1.1894 - val_mse: 2.1895
Epoch 6/10
6670/6670 [==============================] - 1s 190us/sample - loss: 1.3514 - mae: 0.9069 - 

In [14]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [15]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.3247337
Actual val:  7.62578964258

Predicted val:  7.2479277
Actual val:  7.50012615324

Predicted val:  7.18828
Actual val:  6.8122838958

Predicted val:  7.2312145
Actual val:  10.18756112702

Predicted val:  6.571159
Actual val:  7.0327461578

Predicted val:  7.071773
Actual val:  6.2501051276999995

Predicted val:  6.7720666
Actual val:  7.68751907594

Predicted val:  6.6011996
Actual val:  7.054792384

Predicted val:  7.149701
Actual val:  8.24969784404

Predicted val:  7.136253
Actual val:  9.87450471498

